
# Artifact Evaluation

> #### Optimizing Deep Learning Inference via Global Analysis and Tensor Expressions

Optimizing deep neural network (DNN) execution is impor- tant but becomes increasingly difficult as DNN complexity grows. Existing DNN compilers cannot effectively exploit op- timization opportunities across operator boundaries, leaving room for improvement. To address this challenge, we present Souffle, an open-source compiler that optimizes DNN in- ference across operator boundaries. Souffle creates a global tensor dependency graph using tensor expressions, traces data flow and tensor information, and partitions the compu- tation graph into subprograms based on dataflow analysis and resource constraints. Within a subprogram, Souffle per- forms local optimization via semantic-preserving transfor- mations, finds an optimized program schedule, and improves instruction-level parallelism and data reuse. We evaluated Souffle using six representative DNN models on an NVIDIA A100 GPU. Experimental results show that Souffle consis- tently outperforms six state-of-the-art DNN optimizers by 
delivering a geometric mean speedup of up to $3.7\times$ over TensorRT and $7.8\times$ over Tensorflow XLA.

## Preliminaries
This repository showcases the performance evaluation and comparison between `Souffle`(Our work) and the existing state-of-the-art compilers/frameworks(including `XLA`, `TensorRT`, `Rammer`, `Apollo` and `IREE`).

- **XLA(Tensorflow v2.10)**: The TensorFlow XLA compiler
can fuse DNN operators like point-wise and reduction op-
erators and performs optimizations on the fused operator.
Unlike Souffle that performs analysis and optimizations on
TEs, XLA performs analysis on its high-level operators(HLO)

- **TensorRT(v8.2)**:This GPU-vendor-specific framework optimizes the inference of DNNs on NVIDIA GPUs

- **Rammer(v0.4)**:This DNN compiler is also known as NNFu-
sion. It generates a spatial-temporal schedule at compile
time to minimize scheduling overhead and exploit hardware
parallelism through inter- and intra-operator co-scheduling

- **Apollo**:This represents the state-of-the-art fusion framework for inference optimization. Apollo considers both
memory- and compute-bound tensor operators for kernel
fusions and uses hand-crafted rules to exploit parallelism
between independent tensor operators

- **IREE**:The intermediate resentation execution environment (IREE) builds upon the
LLVM MLIR project. IREE is designed to lower DNN
models to MLIR dialects to optimize model inference. IREE
utilizes the linalg dialect to perform the operator fusion,
which supports loop affine fusion optimization and global
analysis.

The metric assesed in this notebook mainly include end-to-end latency, global memory access and number of kernels.

## Important Notes
**A few bash scripts take more than half an hour to complete; Please wait for the results before executing the next one.**
**I will also provide a online jupyter notebook if you are inconvenient or having troubles  to build the environment. The jupyter notebook is run on our server and you can directly access the notebook and run the experiments from the internet**
### Links to The Paper

**For each step, we highlight that the current evaluation is corresponding to which Section or Figure in the submitted paper.**
The main restuls presented in this repository correspond to the submitted paper's Table 3, 4, 5 and Figure 6.


## 1. Experimental Environments Setup

We have setup the experiment environments on our machine.
For more info about how to setup on you local machine, please refer to `README.md`

### Check the status of docker containers

In [1]:
!docker ps

CONTAINER ID   IMAGE                                      COMMAND                  CREATED        STATUS       PORTS                NAMES
71c749bdcc48   souffle-iree:latest                        "/opt/nvidia/nvidia_…"   7 days ago     Up 7 days    6006/tcp, 8888/tcp   crazy_payne
d7bb5016ebae   souffle-tensorrt8.4.1-ubuntu18.04:latest   "/opt/nvidia/nvidia_…"   7 days ago     Up 7 days                         zen_ardinghelli
80eb556e7070   souffle-tvm-0.8:latest                     "/opt/nvidia/nvidia_…"   12 days ago    Up 12 days                        objective_tesla
5a6833d4ccc7   488c0e69d5bc                               "/opt/nvidia/nvidia_…"   12 days ago    Up 12 days                        stoic_margulis
db88060d4da6   cuda_10.24                                 "bash"                   5 weeks ago    Up 2 weeks                        rammer_cuda_fuse
447fe7bfe381   ubuntu:20.04                               "/bin/bash"              3 months ago   Up 2 weeks                  

You should see containers with the following image names:
* souffle-tvm-0.8:latest
* souffle-iree:latest
* souffle-tensorrt8.4.1-ubuntu18.04:latest

### Check the hardware of the server (nvidia A100 GPU)

In [2]:
!nvidia-smi

Tue Oct 24 05:30:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.43.04    Driver Version: 515.43.04    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100 80G...  On   | 00000000:D8:00.0 Off |                    0 |
| N/A   39C    P0    45W / 300W |    118MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. Evaluation
Next, we use four cases to test the end-to-end performance for the baseline compilers and our work.
Each case matches a table or figure in the submitted paper.

we recommend you to run these cases one by one, which reduces the total execution time at most **two hours**.
We also provided a fast mode which re-use the existing profiling data 
to directly print the outputs.

- 2.1 CASE - End-to-end model runtime (Table 3 in Section 8) - around 30 minutes.
- 2.2 CASE - Execution time with Souffle individual optimization (Table 4 in Section 8) - around 30 minutes.
- 2.3 CASE - The number of GPU kernel calls and global memory
data transfer size of the resulting code (Table 5 in Section 8) - around 50 minutes.
- 2.4 CASE - EfficientNet sub-module latency breakdown (Figure 6 in Section 8) - around 10 minutes.

**Log files**

PS: some cases would consume over a half-hour because we have to execute all baselines. Please have a coffee and wait for the output before the subsequent execution.

### 2.1 CASE - End-to-end model runtime (Table 3)
In this case, we compare souffle with five representative state-of-the-art baselines to exploit the end-to-end latency.
We omit Ansor in this case.

| Model       | XLA  | TRT   | Rammer | Apollo | IREE  | Ours |
| ----        | ---- | ----  | ----   | ----   | ----  | ---- |
| BERT        | 2.55 | 1.30  | 2.19   | 3.29   | 2.22  | 1.22 |
| ResNeXt     | 8.91 | 24.82 | 11.69  | 22.80  | 314.8 | 4.43 |
| LSTM        | 10.57| 6.30  | 1.72   | Failed | 16.0  | 0.80 |
| EfficientNet| 2.96 | 1.21  | Falied | 2.3    | 12.33 | 0.66 |
| SwinTrans.  | 6.43 | 1.74  | Falied | 10.78  | 18.1  | 1.55 |
| MMoE        | 0.29 | 0.07  | Falied | 0.049  | 0.088 | 0.014|

**The following commonds reproduce the results of Table 3 in the submiited paper. Please refer to Section8(Page 10) for more details**

Run the experiments:
Note, The args `TRUE` means that we run the workloads rather than read the cached profiling results.

In [8]:
!bash scripts/run_table3.sh TRUE

+++ dirname scripts/run_table3.sh
++ cd scripts
++ pwd
+ script_directory=/home/xiachunwei/Projects/souffle-ae/scripts
+ SOUFFLE_RUN=TRUE
+ '[' -e /home/xiachunwei/Projects/souffle-ae/scripts/../results/table3.csv ']'
+ rm -f /home/xiachunwei/Projects/souffle-ae/scripts/../results/table3.csv
+ touch /home/xiachunwei/Projects/souffle-ae/scripts/../results/table3.csv
++ docker ps -qf ancestor=souffle-tvm-0.8:latest
+ '[' '!' 80eb556e7070 ']'
++ docker ps -qf ancestor=souffle-tvm-0.8:latest
+ souffle_container_id=80eb556e7070
+ echo 80eb556e7070
80eb556e7070
+ docker exec -it -e SOUFFLE_RUN=TRUE 80eb556e7070 /bin/bash -c /workspace/baseline/xla/run_xla.sh
++ cd /workspace/baseline/xla/xla_models
++ export TF_DUMP_GRAPH_PREFIX=/tmp/tf_dump_graph/
++ TF_DUMP_GRAPH_PREFIX=/tmp/tf_dump_graph/
++ export 'TF_XLA_FLAGS=--tf_xla_enable_xla_devices --tf_xla_auto_jit=2 --tf_xla_cpu_global_jit'
++ TF_XLA_FLAGS='--tf_xla_enable_xla_devices --tf_xla_auto_jit=2 --tf_xla_cpu_global_jit'
++ select_latenc

2023-10-23 10:16:35.237025: I tensorflow/compiler/jit/xla_compilation_cache.cc:476] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-10-23 10:16:37.196951: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500
2023-10-23 10:16:38.508243: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-23 10:16:38.600398: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
NoneType: None
NoneType: None
NoneType: None
SKIPPED: /workspace/baseline/xla/xla_models/Swin-Transformer-Tensorflow/swin-trans-nsys.sqlite does not contain NV Tools Extension (NVTX) data.
++ sqlite3 --csv swin-trans-nsys.sqlite 'SELECT names.value AS name, end - start FROM CUPTI_ACTIVITY_KIND_KERNEL AS k JOIN StringIds AS names ON k.demangledName = names.id

-- The C compiler identification is GNU 11.3.0
-- The CXX compiler identification is GNU 11.3.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Found CUDA: /usr/local/cuda (found version "11.8") 
-- Configuring done
-- Generating done
-- Build files have been written to: /root/rammer_generated_models/resnext
-- The C compiler identification is GNU 11.3.0
-- The CXX compiler identification is GNU 11.3.0
-- Detecting C compiler ABI info
-- Detecting C com

Iteration time 13.982304 ms
Iteration time 11.503200 ms
Iteration time 11.466528 ms
Iteration time 11.463072 ms
Iteration time 11.467104 ms
Iteration time 11.471424 ms
Iteration time 11.489281 ms
Iteration time 11.467104 ms
Iteration time 11.482848 ms
Iteration time 11.463936 ms
Iteration time 11.473632 ms
Iteration time 11.464031 ms
Iteration time 11.479392 ms
Iteration time 11.470272 ms
Iteration time 11.459712 ms
Iteration time 11.470368 ms
Iteration time 11.472000 ms
Iteration time 11.469025 ms
Iteration time 11.464224 ms
Iteration time 11.466623 ms
Iteration time 11.483328 ms
Iteration time 11.470753 ms
Iteration time 11.458080 ms
Iteration time 11.448769 ms
Iteration time 11.481120 ms
Iteration time 11.458944 ms
Iteration time 11.452415 ms
Iteration time 11.467008 ms
Iteration time 11.470943 ms
Iteration time 11.478240 ms
Iteration time 11.469025 ms
Iteration time 11.482752 ms
Iteration time 11.481408 ms
Iteration time 11.448192 ms
Iteration time 11.459328 ms
Iteration time 11.47

Iteration time 11.461152 ms
Iteration time 11.474976 ms
Iteration time 11.466048 ms
Iteration time 11.471136 ms
Iteration time 11.464800 ms
Iteration time 11.457696 ms
Iteration time 11.458464 ms
Iteration time 11.457600 ms
Iteration time 11.483904 ms
Iteration time 11.477184 ms
Iteration time 11.457312 ms
Iteration time 11.470464 ms
Iteration time 11.463744 ms
Iteration time 11.453760 ms
Iteration time 11.476992 ms
Iteration time 11.490720 ms
Iteration time 11.485151 ms
Iteration time 11.479392 ms
Iteration time 11.461248 ms
Iteration time 11.473632 ms
Iteration time 11.464320 ms
Iteration time 11.475553 ms
Iteration time 11.463840 ms
Iteration time 11.448576 ms
Iteration time 11.466433 ms
Iteration time 11.464224 ms
Iteration time 11.448576 ms
Iteration time 11.457984 ms
Iteration time 11.465472 ms
Iteration time 11.476032 ms
Iteration time 11.479872 ms
Iteration time 11.474304 ms
Iteration time 11.465088 ms
Iteration time 11.487456 ms
Iteration time 11.486496 ms
Iteration time 11.44

...

==PROF== Profiling "Kernel" - 149: 0%....50%....100% - 1 pass
==PROF== Profiling "Kernel" - 150: 0%....50%....100% - 1 pass
height: 14, width: 14, channel: 512
inline Tensor(shape=[256, 512], op.name=x_roll_permute_matmul)
==PROF== Target process 13250 terminated before first instrumented API call.
==PROF== Target process 13249 terminated before first instrumented API call.
==PROF== Target process 13248 terminated before first instrumented API call.
==PROF== Profiling "swin_transform_auto_tvm_fused..." - 151: 0%....50%....100% - 1 pass
auto_tvm_apply_fused_roll_reshape_permute_reshape_qkv_dense_tensorcore min_latency: 99439.076
2023-10-23 10:28:40,875 ansor_utils.py:60 1696458641.1027641, 1696379585.8763363, False
==PROF== Target process 13257 terminated before first instrumented API call.
==PROF== Target process 13256 terminated before first instrumented API call.
==PROF== Target process 13259 terminated before first instrumented API call.
2023-10-23 10:28:41,061 ansor_utils.py:88 loa

==PROF== Target process 13341 terminated before first instrumented API call.
2023-10-23 10:28:52,980 ansor_utils.py:88 load built module kernel_configs/swin_transformer_fused_window_reverse_roll_add_1_14_14_512_3_7_float16.log.tar
==PROF== Target process 13340 terminated before first instrumented API call.
==PROF== Target process 13342 terminated before first instrumented API call.
==PROF== Profiling "swin_transformer_fused_window..." - 165: 0%....50%....100% - 1 pass
2023-10-23 10:28:53,085 ansor_utils.py:96 kernel_configs/swin_transformer_fused_window_reverse_roll_add_1_14_14_512_3_7_float16.log mean_time 99623.328 us
2023-10-23 10:28:53,089 ansor_utils.py:60 1696456267.4911187, 1696379585.8803368, False
==PROF== Target process 13344 terminated before first instrumented API call.
==PROF== Target process 13343 terminated before first instrumented API call.
==PROF== Target process 13346 terminated before first instrumented API call.
2023-10-23 10:28:53,273 ansor_utils.py:88 load built 

==PROF== Target process 13411 terminated before first instrumented API call.
==PROF== Target process 13420 terminated before first instrumented API call.
==PROF== Target process 13419 terminated before first instrumented API call.
==PROF== Target process 13421 terminated before first instrumented API call.
==PROF== Profiling "swin_transformer_auto_tvm_qeu..." - 180: 0%....50%....100% - 1 pass
TFLOPS: 1.5419183417395435e-07
inline Tensor(shape=[64, 64, 64], op.name=compute)
inline Tensor(shape=[64, 32, 64], op.name=compute)
==PROF== Target process 13424 terminated before first instrumented API call.
==PROF== Target process 13423 terminated before first instrumented API call.
==PROF== Target process 13422 terminated before first instrumented API call.
==PROF== Profiling "swin_transform_auto_tvm_tune_..." - 181: 0%....50%....100% - 1 pass
swin_transformer_main.py:263 [4, 16, 16, 16, 512, 'swin_transform', 'float16']
swin_fused_roll_reshape_qkv_matmul.py:106 x_shape: (256, 32, 16), weight_

inline Tensor(shape=[256, 512], op.name=x_roll_permute_matmul)
==PROF== Target process 13500 terminated before first instrumented API call.
==PROF== Target process 13499 terminated before first instrumented API call.
==PROF== Target process 13498 terminated before first instrumented API call.
==PROF== Profiling "swin_transform_auto_tvm_fused..." - 196: 0%....50%....100% - 1 pass
auto_tvm_apply_fused_roll_reshape_permute_reshape_qkv_dense_tensorcore min_latency: 98410.30900000001
2023-10-23 10:29:15,499 ansor_utils.py:60 1696458641.1027641, 1696379585.8763363, False
==PROF== Target process 13507 terminated before first instrumented API call.
==PROF== Target process 13506 terminated before first instrumented API call.
==PROF== Target process 13509 terminated before first instrumented API call.
2023-10-23 10:29:15,685 ansor_utils.py:88 load built module kernel_configs/swin_transformer_fused_reshape_permute_1_16_16_512_8_16.tar
==PROF== Target process 13508 terminated before first instrume

==PROF== Target process 13573 terminated before first instrumented API call.
==PROF== Target process 13572 terminated before first instrumented API call.
==PROF== Target process 13571 terminated before first instrumented API call.
==PROF== Target process 13580 terminated before first instrumented API call.
==PROF== Target process 13579 terminated before first instrumented API call.
==PROF== Target process 13581 terminated before first instrumented API call.
==PROF== Profiling "swin_transformer_auto_tvm_qeu..." - 210: 0%....50%....100% - 1 pass
TFLOPS: 7.709087064598547e-08
inline Tensor(shape=[32, 64, 64], op.name=compute)
inline Tensor(shape=[32, 32, 64], op.name=compute)
==PROF== Target process 13584 terminated before first instrumented API call.
==PROF== Target process 13583 terminated before first instrumented API call.
==PROF== Target process 13582 terminated before first instrumented API call.
==PROF== Profiling "swin_transform_auto_tvm_tune_..." - 211: 0%....50%....100% - 1 pass

Check for the results:

In [9]:
!cat results/table3.csv

XLA: , 1.7548920000000001, 134.980999, 20.417224, 24.838242, 7896.032, 0.024704

TensorRT: , 2.159784, 24.3297, 8.19212, 1.18884, 2.25791, 0.0769094

Rammer:, 2.052771, 11.552805000000001, 1.798624, failed, failed, failed
Apollo:  4.888844, 13.990651, Failed, 1.787821, 11.389884, 0.046464

IREE: , 13.963836, 337.061494, 8.071082, 4.505689, 22.0568, 0.028896

Souffle: , 1355.52, 4900.0645, 884.32, 802.816, 1945.3119, 23.104



Note that the results in `table3.csv` is a transposed matrix of table 3 in the submmited paper.
### 2.2 CASE - Execution time with Souffle individual optimization (Table 4)
In this case, we present an ablation study by enable the individual optimization one-by-one
to demonstrate the effectiveness of each optmization techniques.
Run the experiments:

In [ ]:
!bash scripts/run_table4.sh TRUE

+++ dirname scripts/run_table4.sh
++ cd scripts
++ pwd
+ script_directory=/home/xiachunwei/Projects/souffle-ae/scripts
+ SOUFFLE_RUN=TRUE
+ '[' -e /home/xiachunwei/Projects/souffle-ae/scripts/../results/table4.csv ']'
+ rm -f /home/xiachunwei/Projects/souffle-ae/scripts/../results/table4.csv
+ touch /home/xiachunwei/Projects/souffle-ae/scripts/../results/table4.csv
++ docker ps -qf ancestor=souffle-tvm-0.8:latest
+ '[' '!' 80eb556e7070 ']'
++ docker ps -qf ancestor=souffle-tvm-0.8:latest
+ souffle_container_id=80eb556e7070
+ echo 80eb556e7070
80eb556e7070
+ souffle_model_path=/workspace/souffle-models/python/models/
+ docker exec -it -e SOUFFLE_RUN=TRUE 80eb556e7070 /bin/bash -c 'cd /workspace/souffle-models/python/models//bert && ./run_ncu_bert.sh'
+ bert_layers=12
+ NCU_ARGS='--metrics dram__bytes_read,gpu__time_duration --clock-control none --target-processes all'
+ '[' -n TRUE ']'
+ '[' TRUE = TRUE ']'
+ ncu --metrics dram__bytes_read,gpu__time_duration --clock-control none --targe

==PROF== Profiling "k1_add_bias" - 219: 0%....50%....100% - 1 pass
==PROF== Profiling "k1_add_bias" - 220: 0%....50%....100% - 1 pass
==PROF== Profiling "k1_add_bias" - 221: 0%....50%....100% - 1 pass
==PROF== Profiling "reshape_384768_1238464" - 222: 0%....50%....100% - 1 pass
==PROF== Profiling "reshape_384768_1238464" - 223: 0%....50%....100% - 1 pass
==PROF== Profiling "reshape_384768_1238464" - 224: 0%....50%....100% - 1 pass
==PROF== Profiling "gemm_k2" - 225: 0%....50%....100% - 1 pass
==PROF== Profiling "softmax" - 226: 0%....50%....100% - 1 pass
==PROF== Profiling "gemm_three_stage" - 227: 0%....50%....100% - 1 pass
==PROF== Profiling "transpose" - 228: 0%....50%....100% - 1 pass
==PROF== Profiling "gemm_three_stage" - 229: 0%....50%....100% - 1 pass
==PROF== Profiling "add_input" - 230: 0%....50%....100% - 1 pass
==PROF== Profiling "add_bias" - 231: 0%....50%....100% - 1 pass
==PROF== Profiling "layernorm" - 232: 0%....50%....100% - 1 pass
==PROF== Profiling "gemm_three_stage

==PROF== Profiling "CatArrayBatchedCopy" - 50: 0%....50%....100% - 1 pass
==PROF== Profiling "vectorized_elementwise_kernel" - 51: 0%....50%....100% - 1 pass
==PROF== Profiling "vectorized_elementwise_kernel" - 52: 0%....50%....100% - 1 pass
==PROF== Profiling "vectorized_elementwise_kernel" - 53: 0%....50%....100% - 1 pass
==PROF== Profiling "vectorized_elementwise_kernel" - 54: 0%....50%....100% - 1 pass
==PROF== Profiling "vectorized_elementwise_kernel" - 55: 0%....50%....100% - 1 pass
==PROF== Profiling "vectorized_elementwise_kernel" - 56: 0%....50%....100% - 1 pass
==PROF== Profiling "DeviceReduceSingleTileKernel" - 57: 0%....50%....100% - 1 pass
==PROF== Profiling "DeviceCompactInitKernel" - 58: 0%....50%....100% - 1 pass
==PROF== Profiling "DeviceSelectSweepKernel" - 59: 0%....50%....100% - 1 pass
[tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., n

==PROF== Target process 15507 terminated before first instrumented API call.
==PROF== Target process 15506 terminated before first instrumented API call.
==PROF== Target process 15508 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv1x1_b..." - 3: 0%....50%....100% - 1 pass
2023-10-23 12:36:26,836 ansor_module.py:46 kernel_configs/resnext_imagenet_50_conv1x1_bn_relu_1_56_56_64_4_1.log, 100800.72300000001
==PROF== Target process 15510 terminated before first instrumented API call.
==PROF== Target process 15509 terminated before first instrumented API call.
==PROF== Target process 15512 terminated before first instrumented API call.
==PROF== Target process 15511 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv1x1_b..." - 4: 0%....50%....100% - 1 pass
==PROF== Target process 15513 terminated before first instrumented API call.
2023-10-23 12:36:27,019 ansor_module.py:46 kernel_configs/resnext_imagenet_50_c

==PROF== Target process 15577 terminated before first instrumented API call.
==PROF== Target process 15576 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv1x1_b..." - 17: 0%....50%....100% - 1 pass
2023-10-23 12:36:29,369 ansor_module.py:46 kernel_configs/resnext_imagenet_50_conv1x1_bn_relu_1_56_56_64_4_1.log, 98121.392
==PROF== Target process 15578 terminated before first instrumented API call.
==PROF== Target process 15580 terminated before first instrumented API call.
==PROF== Target process 15579 terminated before first instrumented API call.
==PROF== Target process 15582 terminated before first instrumented API call.
==PROF== Target process 15581 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv1x1_b..." - 18: 0%....50%....100% - 1 pass
2023-10-23 12:36:29,549 ansor_module.py:46 kernel_configs/resnext_imagenet_50_conv1x1_bn_relu_1_56_56_64_4_1.log, 99450.52
==PROF== Target process 15583 terminated

...

==PROF== Target process 30722 terminated before first instrumented API call.
==PROF== Target process 30721 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv1x1_b..." - 3046: 0%....50%....100% - 1 pass
2023-10-23 12:45:42,295 ansor_module.py:46 kernel_configs/resnext_imagenet_50_conv1x1_bn_relu_1_14_14_1024_16_1.log, 101538.541
==PROF== Target process 30723 terminated before first instrumented API call.
==PROF== Target process 30725 terminated before first instrumented API call.
==PROF== Target process 30724 terminated before first instrumented API call.
==PROF== Target process 30727 terminated before first instrumented API call.
==PROF== Target process 30726 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv1x1_b..." - 3047: 0%....50%....100% - 1 pass
2023-10-23 12:45:42,480 ansor_module.py:46 kernel_configs/resnext_imagenet_50_conv1x1_bn_relu_1_14_14_1024_16_1.log, 101517.138
==PROF== Target process 307

==PROF== Target process 30790 terminated before first instrumented API call.
==PROF== Target process 30789 terminated before first instrumented API call.
==PROF== Target process 30792 terminated before first instrumented API call.
==PROF== Target process 30791 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv1x1_b..." - 3060: 0%....50%....100% - 1 pass
2023-10-23 12:45:44,901 ansor_module.py:46 kernel_configs/resnext_imagenet_50_conv1x1_bn_relu_1_14_14_1024_16_1.log, 102751.31
==PROF== Target process 30793 terminated before first instrumented API call.
==PROF== Target process 30795 terminated before first instrumented API call.
==PROF== Target process 30794 terminated before first instrumented API call.
==PROF== Target process 30797 terminated before first instrumented API call.
==PROF== Target process 30796 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv1x1_b..." - 3061: 0%....50%....100% - 1 pass
==

==PROF== Target process 30927 terminated before first instrumented API call.
==PROF== Target process 30926 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv1x1_b..." - 3087: 0%....50%....100% - 1 pass
==PROF== Target process 30928 terminated before first instrumented API call.
2023-10-23 12:45:49,921 ansor_module.py:46 kernel_configs/resnext_imagenet_50_conv1x1_bn_relu_1_14_14_1024_16_1.log, 100562.539
==PROF== Target process 30930 terminated before first instrumented API call.
==PROF== Target process 30929 terminated before first instrumented API call.
==PROF== Target process 30932 terminated before first instrumented API call.
==PROF== Target process 30931 terminated before first instrumented API call.
==PROF== Target process 30933 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv1x1_b..." - 3088: 0%....50%....100% - 1 pass
2023-10-23 12:45:50,107 ansor_module.py:46 kernel_configs/resnext_imagenet_50_

==PROF== Target process 30995 terminated before first instrumented API call.
==PROF== Target process 30994 terminated before first instrumented API call.
==PROF== Target process 30997 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv3x3_1..." - 3101: 0%....50%....100% - 1 pass
2023-10-23 12:45:52,528 ansor_module.py:46 kernel_configs/resnext_imagenet_50_conv3x3_1_14_14_16_16_3_3.log, 101877.55799999999
==PROF== Target process 30996 terminated before first instrumented API call.
==PROF== Target process 30998 terminated before first instrumented API call.
==PROF== Target process 31000 terminated before first instrumented API call.
==PROF== Target process 30999 terminated before first instrumented API call.
==PROF== Target process 31002 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv3x3_1..." - 3102: 0%....50%....100% - 1 pass
2023-10-23 12:45:52,708 ansor_module.py:46 kernel_configs/resnext_imagenet_50_

==PROF== Target process 31068 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv3x3_1..." - 3115: 0%....50%....100% - 1 pass
2023-10-23 12:45:55,074 ansor_module.py:46 kernel_configs/resnext_imagenet_50_conv3x3_1_14_14_16_16_3_3.log, 101596.41000000002
==PROF== Target process 31067 terminated before first instrumented API call.
==PROF== Target process 31069 terminated before first instrumented API call.
==PROF== Target process 31071 terminated before first instrumented API call.
==PROF== Target process 31070 terminated before first instrumented API call.
==PROF== Target process 31073 terminated before first instrumented API call.
==PROF== Profiling "resnext_imagenet_50_conv3x3_1..." - 3116: 0%....50%....100% - 1 pass
==PROF== Target process 31072 terminated before first instrumented API call.
2023-10-23 12:45:55,254 ansor_module.py:46 kernel_configs/resnext_imagenet_50_conv3x3_1_14_14_16_16_3_3.log, 101457.615
==PROF== Target process 31074 termi

Check for the results:

In [ ]:
cat results/table4.csv

### 2.3 CASE - The number of GPU kernel calls and global memory data transfer size of the resulting code (Table 5)
In this case, We compare Souffle with STOA works about the number of GPU kernel calls and global memory data transfer size (M bytes).
Less number of GPU kernel calls and global memory data transfer size leads to better
fusion results.

Run the experiments:

In [7]:
!bash scripts/run_table5.sh

+++ dirname scripts/run_table5.sh
++ cd scripts
++ pwd
+ script_directory=/home/xiachunwei/Projects/souffle-ae/scripts
+ SOUFFLE_RUN=
+ table5_path=/home/xiachunwei/Projects/souffle-ae/scripts/../results/table5.csv
+ '[' -e /home/xiachunwei/Projects/souffle-ae/scripts/../results/table5.csv ']'
+ rm -f /home/xiachunwei/Projects/souffle-ae/scripts/../results/table5.csv
+ touch /home/xiachunwei/Projects/souffle-ae/scripts/../results/table5.csv
++ docker ps -qf ancestor=souffle-tvm-0.8:latest
+ '[' '!' 80eb556e7070 ']'
++ docker ps -qf ancestor=souffle-tvm-0.8:latest
+ souffle_container_id=80eb556e7070
+ echo 'Souffle docker running: ' 80eb556e7070
Souffle docker running:  80eb556e7070
++ docker ps -qf ancestor=souffle-tensorrt8.4.1-ubuntu18.04:latest
+ '[' '!' d7bb5016ebae ']'
++ docker ps -qf ancestor=souffle-tensorrt8.4.1-ubuntu18.04:latest
+ tensorrt_container_id=d7bb5016ebae
+ echo 'TensorRT docker running:'
TensorRT docker running:
+ docker exec -it -e SOUFFLE_RUN= d7bb5016ebae /bin/

+ SOUFFLE_SWIN_TRANS_MEM=307.4444885253906
++ wc -l ncu-souffle_swin_trans_O4-dram_bytes_read.csv
++ awk '{ print $1 }'
+ SOUFFLE_SWIN_TRANS_NUM_KERNELS=226
+ cd /workspace/souffle-models/python/models/mmoe
+ NAME=souffle_mmoe_O3
+ '[' -n '' ']'
++ python3 ../../extract_ncu_cuda_mem_read.py ncu-souffle_mmoe_O3.csv
+ SOUFFLE_MMoE_MEM=0.13850000500679016
++ wc -l ncu-souffle_mmoe_O3-dram_bytes_read.csv
++ awk '{ print $1 }'
+ SOUFFLE_MMoE_NUM_KERNELS=1
+ cd /workspace/souffle-models/python/models
+ echo 'Souffle number of kernels:,' 24, 109, 1, 66, 226, 1 table5_souffle.csv
+ echo 'Souffle memory read:,' 229.31865692138672, 471.08404541015625, 21.12396812438965, 71.24569702148438, 307.4444885253906, 0.13850000500679016 table5_souffle.csv
+ docker exec -it -e SOUFFLE_RUN= 80eb556e7070 /bin/bash -c 'cd /workspace/souffle-models/python/models && cat table5_souffle.csv'


Check for the results:

In [6]:
!cat results/table5.csv

TensorRT number of kernels:, 156, 3588, 1942, 180, 284, 21

TensorRT: memory read (MBytes):, 753.7812194824219, 822.011474609375, 1101.43701171875, 98.48973083496094, 664.1043090820312, 0.1314999908208847

Apollo number of kernels:, 317, 2551, Failed, 314, 1007, 12

Apollo memory read:, 934.2015991210938, 572.5850830078125, Failed, 168.33407592773438, 1209.5137939453125, 0.14374999701976776

cat: table5_souffle.csv: No such file or directory



## 2.4 CASE - EfficientNet sub-module latency breakdown (Figure 6)
Run the experiments:

In [22]:
!bash scripts/run_figure6.sh

+++ dirname scripts/run_figure6.sh
++ cd scripts
++ pwd
+ script_directory=/home/xiachunwei/Projects/souffle-ae/scripts
+ SOUFFLE_RUN=
+ '[' -e /home/xiachunwei/Projects/souffle-ae/scripts/../results/table3.csv ']'
+ rm -f /home/xiachunwei/Projects/souffle-ae/scripts/../results/table3.csv
+ touch /home/xiachunwei/Projects/souffle-ae/scripts/../results/table3.csv
++ docker ps -qf ancestor=souffle-tvm-0.8:latest
+ '[' '!' 80eb556e7070 ']'
++ docker ps -qf ancestor=souffle-tvm-0.8:latest
+ souffle_container_id=80eb556e7070
+ echo 80eb556e7070
80eb556e7070
+ docker exec -it -e SOUFFLE_RUN= 80eb556e7070 /bin/bash -c /workspace/souffle-models/cpp/efficientnet/run_efficient_se_module_unittest.sh
932.83203
70.24000000000001 57.632 34.624 26.528
34.08 23.488 19.776 14.048
34.816 24.928 20.384 15.84
26.848 17.248 16.736 12.352
26.111999999999995 17.695999999999998 17.44 13.472
24.831999999999997 15.36 18.304 13.312
25.439999999999998 16.544 17.952 13.696
27.136 17.472 20.032 14.368
26.752 17.183

Check for the results:


In [23]:
from IPython.display import Image
Image("results/efficientnet-se-module-latency-ours.png")

ValueError: Cannot embed the 'eps' image format


The image will be saved in `results/efficientnet-se-module-latency-ours.pdf`.
Please compare it with `Figure6`(on page 11) in the submitted paper.

## Contact
If there are any questions or suggestion, please feel free to drop me an email (scscx@leeds.ac.uk). Many thanks for your feedback!


-----
# The end of this Artifact Evaluation
-----

#### Many thanks for your review, time and efforts on this artifact evaluation.  <br> Many thanks for your understanding and bearing with some inconveniences on this notebook. 
